In [ ]:
cd /content/drive/My\ Drive/T-brain/data

[Errno 2] No such file or directory: '/content/drive/My Drive/T-brain/data'
/content


In [ ]:
ls

sample_data/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd

path = "/content/drive/My Drive/T-brain/data/train.csv"
train_data = pd.read_csv(path)
path = "/content/drive/My Drive/T-brain/data/test.csv"
valid_data = pd.read_csv(path)

In [ ]:
train_data.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,0,5,N,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,0,0,N,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,0,5,N,0,6,152458.0,457,59333,N,0,102,0,483434


In [ ]:
valid_data.head(3)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0,592489
1,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0,592452
2,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,100,170013.0,457,59360,N,0,102,0,590212


In [ ]:
print(train_data.shape,'\n',valid_data.shape)

(1521787, 23) 
 (421665, 22)


In [ ]:
'''
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
'''

'\nimport pandas as pd\nimport numpy as np\n%matplotlib inline\nimport matplotlib.pyplot as plt\nimport matplotlib.lines as mlines\nfrom mpl_toolkits.mplot3d import Axes3D\nimport seaborn as sns\nfrom sklearn.model_selection import train_test_split, learning_curve\nfrom sklearn.metrics import average_precision_score\nfrom xgboost.sklearn import XGBClassifier\nfrom xgboost import plot_importance, to_graphviz\nimport warnings\nwarnings.filterwarnings("ignore", category=DeprecationWarning)\n'

# FEATURE ENGINERING (F.....)

In [ ]:
#UNDER SAMPLING FOR IMBALANCE DATA
import numpy as np

delete_size = sum(train_data.fraud_ind == 0)-sum(train_data.fraud_ind == 1)
temp_index=(train_data[train_data['fraud_ind'] == 0]).index.tolist()
resample_index = np.random.choice(temp_index, delete_size, replace=False)
train_data_resample=train_data.drop(train_data.index[resample_index])
train_data_resample.shape


(40710, 23)

In [ ]:
print(train_data_resample.isnull().values.any())
print(valid_data.isnull().values.any())

True
True


In [ ]:
train_data_resample['flbmk'].fillna(value='NaN', inplace=True)
train_data_resample['flg_3dsmk'].fillna(value='NaN', inplace=True)
valid_data['flbmk'].fillna(value='NaN', inplace=True)
valid_data['flg_3dsmk'].fillna(value='NaN', inplace=True)

In [ ]:
train_data_feature=train_data_resample.drop(columns=['fraud_ind'])
#train_data_feature=train_data[['conam','stocn','stscd','csmcu']]
train_data_target=train_data_resample['fraud_ind']

In [ ]:
#想個辦法看看類別型資料會增維到什麼地步 各特徵有幾種樣態
train_data_feature['stscd'].astype(object).describe()

count     40710
unique        4
top           0
freq      34837
Name: stscd, dtype: int64

In [ ]:
categorical_feature=train_data_feature.columns.difference(['acqic','bacno','cano','conam','iterm','locdt','loctm','mchno','scity','txkey'])
#categorical_feature=['']

In [ ]:
train_data_feature=pd.get_dummies(train_data_feature,columns=categorical_feature)
train_data_feature.head(1)

,acqic,bacno,cano,conam,iterm,locdt,loctm,mchno,scity,txkey,contp_0,contp_2,contp_3,contp_4,contp_5,contp_6,csmcu_0,csmcu_3,csmcu_4,csmcu_10,csmcu_12,csmcu_13,csmcu_14,csmcu_18,csmcu_20,csmcu_21,csmcu_22,csmcu_23,csmcu_24,csmcu_25,csmcu_26,csmcu_28,csmcu_29,csmcu_30,csmcu_31,csmcu_32,csmcu_35,csmcu_38,csmcu_40,csmcu_41,...,stocn_45,stocn_46,stocn_47,stocn_48,stocn_49,stocn_50,stocn_52,stocn_53,stocn_55,stocn_56,stocn_61,stocn_62,stocn_63,stocn_68,stocn_69,stocn_73,stocn_75,stocn_76,stocn_78,stocn_80,stocn_83,stocn_85,stocn_87,stocn_89,stocn_91,stocn_92,stocn_93,stocn_94,stocn_95,stocn_96,stocn_98,stocn_101,stocn_102,stocn_104,stocn_106,stocn_107,stscd_0,stscd_1,stscd_2,stscd_3
12,6883,40691,47685,513.8,0,5,172346.0,10045,0,482472,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [ ]:
#onehotencoder = OneHotEncoder(categorical_features = [0])
#data_str_ohe=onehotencoder.fit_transform(data_le).toarray()

#from sklearn.preprocessing import OneHotEncoder
#tempdata=OneHotEncoder(categorical_features = [1,2,3]).fit(train_data_feature)

In [ ]:
from sklearn import preprocessing

QTrans = preprocessing.QuantileTransformer()
train_data_feature_Q = QTrans.fit_transform(train_data_feature)


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#选择K个最好的特征，返回选择特征后的数据
#selection=SelectKBest(chi2, k=300).fit(train_data_feature_Q,train_data_target)
#train_data_feature.columns[selection.get_support()]

In [ ]:
train_data_feature_S=SelectKBest(chi2, k=50).fit_transform(train_data_feature,train_data_target)

In [ ]:
from sklearn.decomposition import PCA

#主成分分析法，返回降维后的数据(unsupervise)
#参数n_components为主成分数目
PCAdata=PCA(n_components=20).fit_transform(train_data_feature_S)

In [ ]:
len(PCAdata[0])

20

In [ ]:
#null_columns=train_data.columns[train_data.isnull().any()]
#train_data[null_columns].isnull().sum()

#ull_columns=test_data.columns[test_data.isnull().any()]
#test_data[null_columns].isnull().sum()

In [ ]:
#train_data.corr()

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 1,'conam'])

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 0,'conam'])

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 0,'stocn'])
#stocn	stscd

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 1,'stocn'])
#stocn	stscd

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 0,'stscd'])

In [ ]:
#plt.figure(figsize=(15,5))
#plt.hist(train_data.loc[train_data['fraud_ind'] == 1,'stscd'])

In [ ]:
########################## JUST FOR TESTING ,NO PRACTICAL USE ######################################

from sklearn.model_selection import train_test_split

tempcolumn=train_data_resample[['fraud_ind']].replace([0,1],[1,0]) #pick list[0,1] -> change list[1,0]
train_data_target=pd.concat([tempcolumn,train_data_resample[['fraud_ind']]],axis=1)

#target=pd.get_dummies(data=target,columns='fraud_ind')
                                                    #PCAdata
X_train, X_test, y_train, y_test = train_test_split(train_data_feature_S, train_data_target, test_size=0.2, random_state=123)

# TENSORFLOW DNN (suck!)

In [ ]:
import tensorflow as tf
tf.random.set_random_seed(123)
# Parameters
learning_rate = 0.001
training_epochs = 5000
batch_size = 100
display_step = 500

# Network Parameters
n_hidden_1 = 90 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
n_input = 50 
n_classes = 2 # MNIST total classes (0-9 digits)
STDDEV = 0.3
#ACTVATION=tf.nn.tanh
DROP_OUT=0.1

In [ ]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
dropout_rate = tf.placeholder(tf.float32)



In [ ]:
def multilayer_perceptron(_X, _weights, _biases, dropout_rate):
    layer1 = tf.nn.dropout( tf.nn.tanh(tf.add(tf.matmul(_X, _weights['w1']), _biases['b1'])), dropout_rate)
    layer2 = tf.nn.dropout( tf.nn.tanh(tf.add(tf.matmul(layer1, _weights['w2']), _biases['b2'])), dropout_rate)
    #layer3 = tf.nn.dropout( ACTVATION(tf.add(tf.matmul(layer2, _weights['w3']), _biases['b3'])), dropout_rate)
    out = tf.nn.sigmoid(tf.add(tf.matmul(layer2, _weights['wout']), _biases['bout']))
    return out

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    #'w3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'wout': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=STDDEV))                                  
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    #'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'bout': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# Construct model
logits = multilayer_perceptron(X, weights, biases, dropout_rate)

#Confusion matrix
confmat=tf.confusion_matrix(tf.argmax(logits, 1),tf.argmax(Y, 1),num_classes=2)


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(36639, 50) (4071, 50) (36639, 2) (4071, 2)


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)

In [ ]:
with tf.Session() as sess:
  sess.run(init)
  for train_index, test_index in kf.split(train_data_feature_S):
    X_train, X_test = train_data_feature_S[train_index], train_data_feature_S[test_index]
    y_train, y_test = train_data_target.iloc[train_index], train_data_target.iloc[test_index]

    # Training cycle
    for epoch in range(training_epochs):
        TrainTP,TrainFP,TrainFN,TestTP,TestFP,TestFN=0,0,0,0,0,0
        
        # Loop over all batches
        
            
          # Run optimization op (backprop) and cost op (to get loss value)
        _= sess.run([train_op], feed_dict={X: X_train,Y: y_train,dropout_rate:DROP_OUT})

        # Display logs per epoch step
        if (epoch+1) % (display_step) == 0:
            confTe,lossTe = sess.run([confmat,loss_op], feed_dict={X: X_test,Y: y_test,dropout_rate: DROP_OUT})
            #TrainTP,TrainFP,TrainFN=confTr[1,1],confTr[0,1],confTr[1,0]
            TestTP,TestFP,TestFN=confTe[1,1],confTe[0,1],confTe[1,0]
            #FscoreTrain = 2*TrainTP/(2*TrainTP+TrainFP+TrainFN or not 0) #'or not' Avoid ZeroDivisionError
            FscoreTest = 2*TestTP/(2*TestTP+TestFP+TestFN or not 0)
            print("Epoch:", '%04d' % (epoch+1),"loss Test=",'%.4f' %lossTe, "FscoreTest=",'%.4f' %(FscoreTest))
            #print(TrainTP,' ',TrainFP,' ',TrainFN,' ',confTr[0,0])

    # Test model
    #pred = tf.nn.softmax(logits)  # Apply softmax to logits
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    #print("Accuracy:",  '%.4f' % (accuracy.eval({X: X_test, Y: y_test})))
  print('-----------------------------------------------')  
sess.close()

Epoch: 0500 loss Test= 0.6938 FscoreTest= 0.6304
Epoch: 1000 loss Test= 0.6924 FscoreTest= 0.5836
Epoch: 1500 loss Test= 0.6923 FscoreTest= 0.5753
Epoch: 2000 loss Test= 0.6869 FscoreTest= 0.5675
Epoch: 2500 loss Test= 0.6853 FscoreTest= 0.5724
Epoch: 3000 loss Test= 0.6788 FscoreTest= 0.6260
Epoch: 3500 loss Test= 0.6760 FscoreTest= 0.6308
Epoch: 4000 loss Test= 0.6734 FscoreTest= 0.6500
Epoch: 4500 loss Test= 0.6735 FscoreTest= 0.6582
Epoch: 5000 loss Test= 0.6717 FscoreTest= 0.6611
Epoch: 0500 loss Test= 0.6706 FscoreTest= 0.6565
Epoch: 1000 loss Test= 0.6721 FscoreTest= 0.6568
Epoch: 1500 loss Test= 0.6708 FscoreTest= 0.6551
Epoch: 2000 loss Test= 0.6728 FscoreTest= 0.6547
Epoch: 2500 loss Test= 0.6714 FscoreTest= 0.6587
Epoch: 3000 loss Test= 0.6734 FscoreTest= 0.6532
Epoch: 3500 loss Test= 0.6725 FscoreTest= 0.6596
Epoch: 4000 loss Test= 0.6725 FscoreTest= 0.6602
Epoch: 4500 loss Test= 0.6730 FscoreTest= 0.6584
Epoch: 5000 loss Test= 0.6702 FscoreTest= 0.6622
Epoch: 0500 loss Tes

KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

In [ ]:
#the following code is shit!!!!!

In [ ]:
import tensorflow as tf
tf.random.set_random_seed(123)

In [ ]:
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['stocn','stscd']
NUMERIC_COLUMNS = ['conam']

def one_hot_cat_column(feature_name, vocab):
  return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = train_data[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X=train_data[['conam','stocn','stscd']]
y=train_data[['fraud_ind']]



In [ ]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y.loc[train_index], y.loc[test_index]

In [ ]:

classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
)

classifier.train(
     input_fn=(X),hool=None,1,1
)

eval_result = classifier.evaluate(
    input_fn=(X_test, y_test)
)


# XGBOOST (very slow)

In [ ]:
X=PCAdata
y=train_data_target.to_numpy().ravel()



In [ ]:
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score
kf = KFold(n_splits=10)
classifier = XGBClassifier()

In [ ]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  print(f1_score(y_test, y_pred))
  print('-------------------------------------------')

0.4998757763975155
-------------------------------------------
0.48763070645243567
-------------------------------------------
0.48553770086526576
-------------------------------------------
0.4859437751004016
-------------------------------------------
0.473510773130545
-------------------------------------------
0.4683252115927161
-------------------------------------------
0.48612538540596095
-------------------------------------------
0.48734339043722835
-------------------------------------------
0.48992089818831336
-------------------------------------------
0.47230169050715215
-------------------------------------------


In [ ]:
print('i am a miserable piese of shit...')

i am a miserable piese of shit...


In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
